In [2]:
import torch
evaluate_order = ["blackhole_var5_base", "disflooding_var5_base", "worstparent_var5_base", "localrepair_var5_base"]

for scenario in evaluate_order:
    print(f"Evaluating scenario: {scenario}")
    # Add your evaluation code here
    model = torch.load(f"/proj/sourasb-220503/IoT_attack_CL_IDS/models/exp_no_1_LSTM_WCL_random/best_model_after_{scenario}.pt")
    print(model)
    model.eval()


Evaluating scenario: blackhole_var5_base


RuntimeError: CUDA error: CUDA-capable device(s) is/are busy or unavailable
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
